In [1]:
import fastf1
import pandas as pd
import numpy as np

In [2]:
session = fastf1.get_session(year=2023, identifier='Race', gp='Austria')
session.load(laps=True, telemetry=False, weather=False, messages=False)

req         WARNING 	DEFAULT CACHE ENABLED! (195.85 MB) /Users/germanbulavkin/Library/Caches/fastf1
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '4', '14', '55', '63', '44', '18', '10', '23', '24', '2', '31', '77', '81', '21', '20', '22', '27']


In [8]:
session = fastf1.get_session(year=2023, identifier='Practice 2', gp='Australia')
session.load(laps=True, telemetry=False, weather=True, messages=False)

req         WARNING 	DEFAULT CACHE ENABLED! (140.74 MB) /Users/germanbulavkin/Library/Caches/fastf1


core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.4.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 2
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 2)
req            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['1', '10', '11', '14', '16', '18', '2', '20', '21', '22', '23', '24', '27', '31', '4', '44', '55', '63', '77', '81']


In [6]:
formatted_laps = session.laps[[
                    "Driver",
                "Team",
                "LapNumber",
                "LapTime",
                "Sector1Time",
                "Sector2Time",
                "Sector3Time",
                "SpeedI1",
                "SpeedI2",
                "SpeedFL",
                "IsPersonalBest",
                "Stint",
                "TyreLife",
                "Position",
                "Compound",
]]

formatted_laps.rename(columns={"SpeedI1": "ST1", "SpeedI2": "ST2", "SpeedFL": "ST3"}, inplace=True)
formatted_laps["LapTime"].round(3)
formatted_laps["Sector1Time"].round(3)
formatted_laps["Sector2Time"].round(3)
formatted_laps["Sector3Time"].round(3)

/var/folders/0h/_ft064n17r98nzj1w0nz7dq00000gn/T/ipykernel_1403/160505874.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_laps.rename(columns={"SpeedI1": "ST1", "SpeedI2": "ST2", "SpeedFL": "ST3"}, inplace=True)


In [8]:
lec_telemetry = laps.pick_driver('16').pick_lap(20).get_telemetry()
ver_telemetry = laps.pick_driver('4').pick_lap(20).get_telemetry()

filter_columns = ['Time', 'Speed', 'Distance', 'RelativeDistance']

lec_telemetry = lec_telemetry[filter_columns]
ver_telemetry = ver_telemetry[filter_columns]

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


In [9]:
def get_interpolated_time_at_distance(telemetry, relative_distance_point: float | int) -> float:
    threshold_index = 0
    
    RELATIVE_DISTANCE_COLUMN_INDEX = 3
    TIME_INDEX = 0

    TOLERANCE = 0.0005

    for index, row in enumerate(telemetry.values):
        if row[RELATIVE_DISTANCE_COLUMN_INDEX] <= relative_distance_point:
            threshold_index = index
        else: 
            break
    
    if np.abs(telemetry.values[threshold_index][RELATIVE_DISTANCE_COLUMN_INDEX]-relative_distance_point) < TOLERANCE: 
        return telemetry.values[threshold_index][TIME_INDEX]

    lower_than_avg_data_row = telemetry.values[threshold_index]
    higher_than_avg_data_row = telemetry.values[threshold_index + 1]
    
    lo_time = lower_than_avg_data_row[TIME_INDEX]
    hi_time = higher_than_avg_data_row[TIME_INDEX]
    
    lo_distance = lower_than_avg_data_row[3]
    hi_distance = higher_than_avg_data_row[3]
    
    diff = hi_distance - lo_distance

    lo_diff = relative_distance_point - lo_distance
    hi_diff = hi_distance - relative_distance_point

    return (lo_diff / diff) * lo_time + (hi_diff / diff) * hi_time

In [10]:
def get_deltas(telemetry, telemetry_compared_to):
    deltas = [] 

    RELATIVE_DISTANCE_COLUMN_INDEX = 3
    TIME_INDEX = 0

    for row in telemetry.values:
        rel_distance = row[RELATIVE_DISTANCE_COLUMN_INDEX]
        compared_time = row[TIME_INDEX]
 
        interpolated_time = get_interpolated_time_at_distance(telemetry=telemetry_compared_to, relative_distance_point=rel_distance)
        deltas.append([rel_distance, (compared_time-interpolated_time).total_seconds()])
                      
    return deltas 

In [11]:
deltas = get_deltas(ver_telemetry, lec_telemetry)
deltas

IndexError: index 731 is out of bounds for axis 0 with size 731

In [12]:
laps['IsOutlap'] = pd.isna(laps['Sector1Time'])
laps['IsInlap'] = pd.isna(laps['Sector3Time'])